### This Notebook will use a formula to determine the reccomended KVA for a transformer when it is replaced
The formula uses customer count and customer consumption history downsteam of each transformer. When Electric Trouble Crews replace a transformer the reccomended size will available when an Engineer is not (for example after hours or weekends). This will help ensure the right size transformer is used based on consumption history, and not merley just areplacement that could be under-sized or over-sized.
The second half of the Notebook explores the electric consumption data with a [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#spatialdataframe) using the customers table modifed in the first half.


In [1]:
import arcpy
#import numpy
#import pandas as pd
from datetime import datetime

In [2]:
gdb = r"C:\Users\friendde\Documents\ArcGIS\Projects\ReplaceKVA\ReplaceKVA.gdb"
txSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\Electric\main.eTransformerBank"
txDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\ReplaceKVA\ReplaceKVA.gdb\Electric\eTransformerBank"
svcPntDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\ReplaceKVA\ReplaceKVA.gdb\Electric\eServicePoint"
custAcctSource = r"C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCUSTOMERACCOUNT"
custAcctDest = r"C:\Users\friendde\Documents\ArcGIS\Projects\ReplaceKVA\ReplaceKVA.gdb\eCUSTOMERACCOUNT"
txAddFlds = ["kWDSum","CF","kWDSumXCF","kWDSumXCF_A","kWDSumXCF_B","kWDSumXCF_C"]
txFlds = ["GLOBALID","kWDSum","CF","kWDSumXCF","CUSTOMERCOUNT","MAXCUSTOMERCONSUMPTION_A","MAXCUSTOMERCONSUMPTION_B","MAXCUSTOMERCONSUMPTION_C","DEVICEID","FEEDERID","FEEDERID2","INSTALLATIONDATE","PHASEDESIGNATION","SUBTYPE","RATEDKVA_A","RATEDKVA_B","RATEDKVA_C","STRUCTUREID","FACILITYID","AVGCUSTOMERCONSUMPTION_A","AVGCUSTOMERCONSUMPTION_B","AVGCUSTOMERCONSUMPTION_C","PURPOSE","STREETADDRESS","eSupportStructure_GlobalID","eSurfaceStructure_GlobalID"]
svcPntFlds = ["OID@","GLOBALID","eTransformerBank_GLOBALID","FEEDERID","PHASEDESIGNATION","SUBTYPE","CUSTOMERCOUNT","AVGCUSTOMERCONSUMPTION","MAXCUSTOMERCONSUMPTION","STREETADDRESS","eSupportStructure_GLOBALID"]
custAcctFlds = ["eServicePoint_GlobalID","MAXCONSUMPTION"]

- For each customer downstream of a transformer, obtain Estimated Demand using summer or winter demand table, translate kwH (key) to kwD (vale). 
- Sum the kwD 
- Using coincidence factor dictionary, number of customers downstream of transformer (key) to obtain coincidence factor (value). - Multiply coincidence factor by sum of kwD
- Result is minimum kvA transformer
 
After running Accumulate Upstream (seperate C# program run against GIS that traces geometric network) the number of customers and consumption is available on each transformer. Obtain CF using number of customers on transformer, convert maxconsumption to kwD. Multiply kwD by CF. Result is minimum kvA transformer.

In [3]:
if datetime.today().month >= 5 <= 9:
    #print("Using Summer Peak")
    summerPeak = True
    estimatedDemand = {50:2.40,100:2.67,150:2.94,200:3.20,250:3.46,300:3.72,350:3.98,400:4.23,450:4.48,500:4.73,550:4.97,600:5.21,
                    650:5.45,700:5.68,750:5.92,800:6.15,850:6.37,900:6.60,950:6.82,1000:7.03,1050:7.25,1100:7.46,1150:7.67,
                    1200:7.88,1250:8.08,1300:8.28,1350:8.48,1400:8.67,1450:8.86,1500:9.05,1550:9.24,1600:9.42,1650:9.60,
                    1700:9.78,1750:9.95,1800:10.12,1850:10.29,1900:10.46,1950:10.62,2000:10.78,2050:10.94,2100:11.10,2150:11.26,
                    2200:11.41,2250:11.57,2300:11.73,2350:11.89,2400:12.05,2450:12.21,2500:2.36,2550:2.52,2600:12.68,2650:12.84,
                    2700:13.00,2750:13.16,2800:13.31,2850:13.47,2900:13.63,2950:13.79,3000:13.95}
else:
    #print("Using Winter Peak")
    winterPeak = True
    estimatedDemand = {50:1.97,100:2.33,150:2.69,200:3.05,250:3.41,300:3.76,350:4.11,400:4.45,450:4.80,500:5.14,550:5.48,
                       600:5.82,650:6.15,700:6.49,750:6.82,800:7.14,850:7.47,900:7.79,950:8.11,1000:8.43,1050:8.74,1100:9.05,
                       1150:9.36,1200:9.67,1250:9.97,1300:10.27,1350:10.57,1400:10.87,1450:11.16,1500:11.45,1550:11.74,1600:12.03,
                       1650:12.31,1700:12.60,1750:12.87,1800:13.15,1850:13.42,1900:13.70,1950:13.96,2000:14.23,2050:14.45,
                       2100:14.68,2150:14.90,2200:15.13,2250:15.35,2300:15.58,2350:15.80,2400:16.03,2450:16.25,2500:16.48,
                       2550:16.70,2600:16.93,2650:17.15,2700:17.38,2750:17.60,2800:17.83,2850:18.05,2900:18.28,2950:18.50,
                       3000:18.73}

# coincidence factor
cf = {1:1.0,2:0.85,3:0.74,4:0.66,5:0.61,6:0.57,7:0.54,8:0.52,9:0.5,10:0.49,11:0.47,12:0.46,13:0.45,14:0.43,15:0.42,16:0.41,
      17:0.39,18:0.38,19:0.38,20:0.37}


In [4]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True

In [31]:
def sumSvcPnts(_OID,_GID):
    svcGUID = []
    consumption = []
    svcSum = 0
    with arcpy.da.SearchCursor(svcPntDest,svcPntFlds,f"eTransformerBank_GLOBALID = '{_GID}'") as svcPnts:
        for svcPnt in svcPnts:
            print(f'svcPntCon {svcPnt[8]}')
            print(f'eTransformerBank_GLOBALID = {svcPnt[2]}')
            svcGUID.append(GLOBALID)
            print(svcGUID)
    for guid in svcGUID:
        consumption.append(getConsumption(guid))
        print(consumption)
    return sum(consumption)

In [27]:
def getConsumption(svcPntGUID):
    count = 0 
    kwh = [] 
    kwhSum = 0 
    minimumKVA = 0 
    cf = 1 
    with arcpy.da.SearchCursor(custAcctDest,custAcctFlds,f"eServicePoint_GlobalID = '{svcPntGUID}'") as custAccts:
        for custAcct in custAccts:
            kwh.append(custAcct[1])
    return sum(kwh)

In [7]:
# TODO use switch-case
def updateTx(OID,GID,phase,consumption):
    viewTbl = "tcview"+str(OID)
    txCon = []
    txSum = 0
    arcpy.MakeTableView_management(txDest,viewTbl,where_clause=f"GLOBALID = '{GID}'",workspace=gdb)
    if int(arcpy.GetCount_management(viewTbl).getOutput(0)) == 1:
        with arcpy.da.UpdateCursor(txDest,txFlds,f"GLOBALID = '{GID}'") as txPnts:
            #print(f'eTransformerBank_GLOBALID = "{txGUID}"')
            for txPnt in txPnts:
                #print(f'svcPntGUID {svcPnt[0]}')
                if phase == 1: # c phase [7]
                    txCon.append(txPnt[7])
                    txCon.append(consumption)
                    txPnt[7] = sum(txCon)
                if phase == 2: # b phase [6]
                    txCon.append(txPnt[6])
                    txCon.append(consumption)
                    txPnt[6] = sum(txCon)
                if phase == 3: # bc phase [6][7]
                    txCon.append(txPnt[6])
                    txCon.append(consumption/2)
                    txPnt[6] = sum(txCon)
                    txCon = []
                    txCon.append(txPnt[7])
                    txCon.append(consumption/2)
                    txPnt[7] = sum(txCon)
                if phase == 4: # a phase [5]
                    txCon.append(txPnt[5])
                    txCon.append(consumption)
                    txPnt[5] = sum(txCon)
                if phase == 5: # ac phase [5][7]
                    txCon.append(txPnt[5])
                    txCon.append(consumption/2)
                    txPnt[5] = sum(txCon)
                    txCon = []
                    txCon.append(txPnt[7])
                    txCon.append(consumption/2)
                    txPnt[7] = sum(txCon)
                if phase == 6: # ab phase [5][6]
                    txCon.append(txPnt[5])
                    txCon.append(consumption/2)
                    txPnt[5] = sum(txCon)
                    txCon =[]
                    txCon.append(txPnt[6])
                    txCon.append(consumption/2)
                    txPnt[6] = sum(txCon)
                if phase == 7: # abc phase [5][6][7]
                    txCon.append(txPnt[5])
                    txCon.append(consumption/2)
                    txPnt[5] = sum(txCon)
                    txCon =[]
                    txCon.append(txPnt[6])
                    txCon.append(consumption/2)
                    txPnt[6] = sum(txCon)
                    txCon =[]
                    txCon.append(txPnt[7])
                    txCon.append(consumption/2)
                    txPnt[7] = sum(txCon)
                txPnts.updateRow(txPnt)

In [8]:
def getKWD(consumption): 
    for key in sorted(estimatedDemand.keys()):
        if consumption <= key:
            #print(key, estimatedDemand[key])
            return estimatedDemand[key]
def getKWDPhase(consumption):
    for key in sorted(estimatedDemand.keys()):
        if consumption <= key:
            #print(key, estimatedDemand[key])
            return estimatedDemand[key]
def calcKWD(consumption):
    if summerPeak: 
        print(f'Using Summer Peak')
        return 13.95+0.0025(consumption-3000) 
    if winterPeak:
        print(f'Using Winter Peak')
        return 18.70+0.0027(consumption-3000)

def getCF(custCount):
    for key in sorted(cf.keys()):
        if custCount == key:
            #print(key, cf[key])
            return cf[key]
        if custCount >= 20:
            return .37

 def getCustAcct(svcPntGUID):
     count = 0
     kwd = []
     kwdSum = 0
     minimumKVA = 0
     _cf = 1
     with arcpy.da.SearchCursor(custAcctDest,custAcctFlds,f"eServicePoint_GlobalID = '{svcPntGUID}'") as custAccts:
         for custAcct in custAccts:
             if custAcct[3] is None:
                 pass
             else:
                 custAcctCount += 1
                 print(f'MaxConsumption {custAcct[3]} and count is {count}')
                 kwdVal = getKWD(custAcct[3])
                 if kwdVal is None:
                     kwdVal = calcKWD(custAcct[3])
                 kwd.append(kwdVal)
                 #if count > 1:
                 kwdSum = sum(kwd)
                 #print(f'Sum of kwd {kwdSum}')
             _cf = getCF(count)
             if kwdSum == 0:
                 pass
             elif _cf is None:
                 pass
             else:
                 minimumKVA = kwdSum*_cf
             print(f'kwdList: {kwd}')
             print(f'Sum of kwd {kwdSum}')
             print(f'_cf {_cf}')
             print(f'Estimated Load: {minimumKVA}kva')
         return kwdSum, _cf, minimumKVA
     return 0,0,0

 def getKWD(consumption):
     for key in sorted(estimatedDemand.keys()):
         if consumption <= key:
             #print(key, estimatedDemand[key])
             return estimatedDemand[key]
 def getKWDPhase(consumption):
     for key in sorted(estimatedDemand.keys()):
         if consumption <= key:
             #print(key, estimatedDemand[key])
             return estimatedDemand[key]

def calcKWD(consumption):
     if summerPeak:
         print(f'Using Summer Peak')
         return 13.95+0.0025*(consumption-3000)
     if winterPeak:
         print(f'Using Winter Peak')
         return 18.70+0.0027*(consumption-3000)

 def getCF(custCount):
     for key in sorted(cf.keys()):
         if custCount == key:
             #print(key, cf[key])
             return cf[key]
         if custCount >= 20:
             return .37

In [10]:
tblList = arcpy.ListTables()
for tbl in tblList:
    print(tbl)
    arcpy.Delete_management(tbl)
fdsList = arcpy.ListDatasets()
for fds in fdsList:
    print(fds)
    arcpy.Delete_management(fds)

Electric


In [11]:
arcpy.Copy_management(custAcctSource,custAcctDest)
for fld in txAddFlds:
    arcpy.AddField_management(txDest,fld,"DOUBLE")

In [12]:
txMaxFlds = ["MAXCUSTOMERCONSUMPTION_A","MAXCUSTOMERCONSUMPTION_B","MAXCUSTOMERCONSUMPTION_C"]
for txMaxFld in txMaxFlds:
    arcpy.CalculateField_management(txDest,txMaxFld,0)
arcpy.CalculateField_management(svcPntDest,"MAXCUSTOMERCONSUMPTION",0)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\ReplaceKVA\\ReplaceKVA.gdb\\Electric\\eServicePoint'>

In [13]:
expression = "setZero(!MAXCONSUMPTION!)"
codeblock = """
def setZero(maxcon):
    if maxcon is None:
        return 0
    else:
        return maxcon"""
arcpy.CalculateField_management(custAcctDest,"MAXCONSUMPTION",expression,"PYTHON3",codeblock)

<Result 'C:\\Users\\friendde\\Documents\\ArcGIS\\Projects\\ReplaceKVA\\ReplaceKVA.gdb\\eCUSTOMERACCOUNT'>

In [12]:
edit = arcpy.da.Editor(gdb)
edit.startEditing(False, False)
edit.startOperation()

In [32]:
#svcPntFlds = ["OID@","GLOBALID","eTransformerBank_GLOBALID","FEEDERID","PHASEDESIGNATION","SUBTYPE","CUSTOMERCOUNT",
              #"AVGCUSTOMERCONSUMPTION","MAXCUSTOMERCONSUMPTION","STREETADDRESS","eSupportStructure_GLOBALID"]
#txFlds = ["OID@","GLOBALID","kWDSum","CF","kWDSumXCF","CUSTOMERCOUNT","MAXCUSTOMERCONSUMPTION_A","MAXCUSTOMERCONSUMPTION_B",
          #"MAXCUSTOMERCONSUMPTION_C",,"PHASEDESIGNATION","DEVICEID","FEEDERID","FEEDERID2","INSTALLATIONDATE","SUBTYPE",
          #"RATEDKVA_A","RATEDKVA_B","RATEDKVA_C","STRUCTUREID","FACILITYID","AVGCUSTOMERCONSUMPTION_A","AVGCUSTOMERCONSUMPTION_B",
          #"AVGCUSTOMERCONSUMPTION_C","PURPOSE","STREETADDRESS","eSupportStructure_GlobalID","eSurfaceStructure_GlobalID"]
with arcpy.da.UpdateCursor(txDest,txFlds,where_clause="GLOBALID = '{7265559C-7F0D-40E5-95EC-294E8023FE1D}'") as txPnts:
        for txPnt in txPnts:
            count = 0 
            kwh = [] 
            kwhSum = 0
            kwhSum = sumSvcPnts(txPnt[0],txPnt[1])
            print(f"kwh sum: {kwhSum}")

kwh sum: 0


In [ ]:
with arcpy.da.SearchCursor(custAcctDest,custAcctFlds) as accounts:
        for account in accounts:
            print(f'SvcPntGUID {account[2]} Customer Consumption {account[4]}')
            updateSvcPnts(account[0],account[2],account[4])

SvcPntGUID {35931822-0321-4495-9888-0D162F19FBF0} Customer Consumption 1721.0
SvcPntGUID {7F7DCA92-2099-4009-98AD-198C7F1B8584} Customer Consumption 1728.0
SvcPntGUID {A9BF55EA-A956-4B28-8712-C29AB3313452} Customer Consumption 2117.0
SvcPntGUID {52ED8C1B-F820-429E-9524-6F2F34FD3663} Customer Consumption 820.0
SvcPntGUID {8EB5D4D2-82B3-47DA-B2BC-8E7C6CD5B4BB} Customer Consumption 2435.0
SvcPntGUID {B3124681-54C5-47B3-B4BE-87AF7A0E2193} Customer Consumption 1998.0
SvcPntGUID {C85119B6-7A6A-4DA9-9A68-2DA78BDDB6A5} Customer Consumption 2107.0
SvcPntGUID {D3674A61-14C9-4CDD-94B1-DD050C359EB8} Customer Consumption 1157.0
SvcPntGUID {352C442B-A110-481D-A3B6-9ABB5694AF93} Customer Consumption 0.0
SvcPntGUID {6EDA99A9-9EC1-456C-99CB-6E06822A34B9} Customer Consumption 2891.0
SvcPntGUID {36C9A786-DAE3-483E-AAF2-AABBE2B1F664} Customer Consumption 76.0
SvcPntGUID {4E331973-449B-4D51-9888-4BCAD1FB253D} Customer Consumption 0.0
SvcPntGUID {83CB2E47-F5D4-48A4-87D2-2DE22A190A9A} Customer Consumption 37

SvcPntGUID {9F494478-939C-4E6B-96D0-92935B8DD7E2} Customer Consumption 1817.0
SvcPntGUID {2DF3B0AE-C1FA-4804-A99E-4E5C3B08F165} Customer Consumption 1781.0
SvcPntGUID {F88958E9-B605-4381-A9E8-C6FCB3A65AFC} Customer Consumption 1626.0
SvcPntGUID {CE0AC75F-64B8-4101-9709-08CAAD1AF67C} Customer Consumption 1859.0
SvcPntGUID {920CB076-4204-4654-AA29-F910841AAD61} Customer Consumption 1674.0
SvcPntGUID {74C2A395-B206-441D-92B3-93CA08FFC177} Customer Consumption 1210.0
SvcPntGUID {7E9129AB-3EA1-4901-816C-B6CABA71BCB9} Customer Consumption 0.0
SvcPntGUID {0A9498A3-CC32-4764-9C29-516C20147E12} Customer Consumption 2116.0
SvcPntGUID {0F1A0EFF-A488-4D90-A77D-7A83A94F678B} Customer Consumption 2173.0
SvcPntGUID {49B0A037-651C-4885-B5BA-EACBF7A05AB0} Customer Consumption 940.0
SvcPntGUID {E37DA04D-AD60-482A-8249-BF3A46C3D2DF} Customer Consumption 2525.0
SvcPntGUID {3A9AE88F-4F01-46C7-A070-B2105651D018} Customer Consumption 954.0
SvcPntGUID {5A740F7E-AADF-4AE8-832C-E69D69B755F8} Customer Consumptio

SvcPntGUID {27E971D4-C784-4CD2-884D-B80895A5640B} Customer Consumption 1405.0
SvcPntGUID {FD9C466D-1810-4F0F-9596-42E2D63D6595} Customer Consumption 1380.0
SvcPntGUID {FAC8AB09-3C21-48DB-A1F8-D9E3845270A1} Customer Consumption 3675.0
SvcPntGUID {DF601F5F-51E5-4C6B-BE2F-956FF5932C95} Customer Consumption 1400.0
SvcPntGUID {13043470-14A4-4434-BB91-F4F6625D26FC} Customer Consumption 391.0
SvcPntGUID {7ABD182C-D16B-4CEF-9EF6-F781AEA78964} Customer Consumption 1530.0
SvcPntGUID {4E79A7E1-BED3-4BD7-A93D-A98C4623EC3A} Customer Consumption 2840.0
SvcPntGUID {E8F5BFBA-F756-4E06-9DD8-23569877A6F0} Customer Consumption 1984.0
SvcPntGUID {C8DFDE5D-3729-41B9-83F0-DF056638C241} Customer Consumption 1522.0
SvcPntGUID {32FB902D-B63F-4658-B4E7-9080C284E5BE} Customer Consumption 1983.0
SvcPntGUID {134C0226-E4DE-4E8C-A33D-C4F1A918C9BD} Customer Consumption 2324.0
SvcPntGUID {1E51D482-F10B-49F7-B082-E438872C09E1} Customer Consumption 1434.0
SvcPntGUID {38BC2563-3F49-4BA5-8A28-8220BF982C85} Customer Consum

SvcPntGUID {7885C0A5-4656-47D4-9C88-3DCEE28611FD} Customer Consumption 1125.0
SvcPntGUID {F61D22A8-E44A-492F-8321-DE4D778600A0} Customer Consumption 1561.0
SvcPntGUID {4356C732-34CD-409E-8663-0815232B32B3} Customer Consumption 2874.0
SvcPntGUID {2E7D7CE3-8B30-4842-86EC-FF9D460FDF6A} Customer Consumption 1202.0
SvcPntGUID {F003308F-CBEC-4D48-B70D-3EBAB047CA04} Customer Consumption 817.0
SvcPntGUID {1E9B6A44-55D4-4390-9B8E-5E5BE6C6E4CB} Customer Consumption 1923.0
SvcPntGUID {F8D92DA7-25B5-4FCD-BBAB-E96D4619765F} Customer Consumption 2125.0
SvcPntGUID {8843C787-167F-4AD8-8F17-5A0209B6CABF} Customer Consumption 149.0
SvcPntGUID {47C8EA07-5D70-45E6-9C27-4F1C77251DC6} Customer Consumption 2668.0
SvcPntGUID {ACD601E6-F10A-4146-950B-C8FE1B4B60DB} Customer Consumption 2040.0
SvcPntGUID {98CB98F3-8503-4CC9-AFFA-57EDE6175361} Customer Consumption 1153.0
SvcPntGUID {5D03E805-A704-4B24-B9C0-B9AFEF6D82BB} Customer Consumption 1775.0
SvcPntGUID {E9318451-9457-4066-B121-06EBE438C20B} Customer Consump

SvcPntGUID {F64FE75F-668E-485D-AE19-6E20A7E826A5} Customer Consumption 1053.0
SvcPntGUID {B8CBA909-DC00-4104-9EEC-50D957BA603B} Customer Consumption 1744.0
SvcPntGUID {BCC04693-EAF5-428D-B45E-2E8CD114A470} Customer Consumption 1818.0
SvcPntGUID {BDBF5E95-635C-43E3-907A-B16FBE099F59} Customer Consumption 1233.0
SvcPntGUID {D251ECD4-DF49-4EE9-B99A-44364C148F81} Customer Consumption 4616.0
SvcPntGUID {71F804A9-5843-447F-BC41-1CB3E81F9AB6} Customer Consumption 607.0
SvcPntGUID {55C43248-AFC5-4F0E-A870-1AFCC4401549} Customer Consumption 1178.0
SvcPntGUID {FE011452-F185-4D8E-A46C-07C1B3E57B9A} Customer Consumption 1703.0
SvcPntGUID {9DB4C4C5-085D-449A-BE11-359F4D5142B8} Customer Consumption 1223.0
SvcPntGUID {57FD6424-37E9-4AB3-85F0-EC38E2ADF2DA} Customer Consumption 4019.0
SvcPntGUID {65AA4507-2AD9-42B0-A737-271D27D9E1E4} Customer Consumption 2171.0
SvcPntGUID {B3BCFC05-C910-4721-B87D-85A694BBF2F9} Customer Consumption 1644.0
SvcPntGUID {F43B6E50-65A8-4C25-993E-B9A9166788A0} Customer Consum

...

SvcPntGUID {2CE966CE-CCD9-4D63-B6A3-F8F25F2B0C8D} Customer Consumption 349.0
SvcPntGUID {B6AA0495-2CF1-41F9-9ED6-DBBB88D0400C} Customer Consumption 451.0
SvcPntGUID {7BDF3E8F-65E3-458F-80F5-88D1DA35A01A} Customer Consumption 1204.0
SvcPntGUID {50A0E4FA-D44A-477D-9104-18BCEE71005C} Customer Consumption 1394.0
SvcPntGUID {1AF18F1A-45E0-432E-BC00-88B316A04AF1} Customer Consumption 917.0
SvcPntGUID {90580E37-CA1E-4A61-963A-6CB40A43F398} Customer Consumption 1291.0
SvcPntGUID {D6FF909F-35C5-4207-937B-A011E287A8B2} Customer Consumption 1596.0
SvcPntGUID {825E4A02-D709-4423-90C9-C1A24BB796C1} Customer Consumption 1231.0
SvcPntGUID {AC4AD33A-59C7-4B35-BA4A-59D2A420050B} Customer Consumption 1776.0
SvcPntGUID {84E07246-0C76-47CA-BEDD-181DDA1996C4} Customer Consumption 1026.0
SvcPntGUID {2E33856C-2324-44F0-812F-83BDB5474F8A} Customer Consumption 1575.0
SvcPntGUID {BFD0FBED-3A00-462E-87A8-85F8C9B093EA} Customer Consumption 900.0
SvcPntGUID {E3009CCB-2F09-4287-80B5-378F577EDC2B} Customer Consumpti

SvcPntGUID {656D3FF5-D3A6-440A-8B8F-6AA32667D6C7} Customer Consumption 1589.0
SvcPntGUID {529BD840-0F0F-4817-BCE2-551444103764} Customer Consumption 1016.0
SvcPntGUID {9DDCD911-BA5F-4679-87E6-2CC89D743200} Customer Consumption 2339.0
SvcPntGUID {9AE28D8C-B847-43F5-86BC-55289477B245} Customer Consumption 1996.0
SvcPntGUID {52175EC5-A3D3-4236-A139-E8D4551E5E66} Customer Consumption 1860.0
SvcPntGUID {2A5E2532-96D9-450F-AC3E-525D8AE9777A} Customer Consumption 2524.0
SvcPntGUID {2E094DEB-B7B2-460C-89C9-D1820447857D} Customer Consumption 243.0
SvcPntGUID {5B80A548-85E9-469A-AB7B-CA3C857B2CF4} Customer Consumption 1858.0
SvcPntGUID {BA61F1A3-AE40-42C9-BBC5-AB089026AF7F} Customer Consumption 1907.0
SvcPntGUID {C65DE073-CA55-48D3-9C56-61443397BDE3} Customer Consumption 2437.0
SvcPntGUID {FFCB36EC-AE72-4AA2-B7EB-4FC3E72BAC6C} Customer Consumption 1335.0
SvcPntGUID {DB5A46D4-C060-4C27-82E4-6EA780B5B1CC} Customer Consumption 2051.0
SvcPntGUID {DAFED1BB-ED14-4B73-AD13-970EF0970D0D} Customer Consum

SvcPntGUID {0A8E8AB8-AFAD-4FCE-984A-D9DCEFCCB626} Customer Consumption 3057.0
SvcPntGUID {CBF84B58-4CDB-4D1F-B156-DD3549D1E952} Customer Consumption 3485.0
SvcPntGUID {10234F0B-1ED8-43A3-920E-5996D726A92F} Customer Consumption 2980.0
SvcPntGUID {1DEACC96-29B1-4F95-8E48-5707CAF50487} Customer Consumption 840.0
SvcPntGUID {3691E99A-C08F-442B-8194-7AB4D6958323} Customer Consumption 1997.0
SvcPntGUID {C198186D-DC6A-4A49-9AC0-47F84EC8904C} Customer Consumption 32.0
SvcPntGUID {9866E6DE-12BF-41FD-A95B-200A62B1353F} Customer Consumption 2680.0
SvcPntGUID {13EF06BB-C468-44A4-A087-1DDEF937A298} Customer Consumption 3959.0
SvcPntGUID {205D7215-6518-4D9E-82CE-59A3B11220C2} Customer Consumption 1764.0
SvcPntGUID {10FAEDA3-F032-4EA2-BB2F-FEEA89F35A46} Customer Consumption 3759.0
SvcPntGUID {981F55E1-4B27-4771-A811-81B6C3971739} Customer Consumption 1753.0
SvcPntGUID {7546379B-5EF9-4D77-B45B-A4F4C32006FF} Customer Consumption 2380.0
SvcPntGUID {BFE31836-3D9A-4889-9200-E088F77300EC} Customer Consumpt

SvcPntGUID {84E7ABC4-B4C6-417F-A7A1-8A15FF6F29C7} Customer Consumption 3403.0
SvcPntGUID {0E3E5590-A0C9-4D46-97D1-18C301749AB1} Customer Consumption 2696.0
SvcPntGUID {58A1E0CF-C801-4084-B561-6B8C40550B97} Customer Consumption 1760.0
SvcPntGUID {5EF741CD-150A-4340-9D3F-988CF063EB85} Customer Consumption 1043.0
SvcPntGUID {1D58E0F4-040F-4845-A8E2-70CBBCE0F4E7} Customer Consumption 2622.0
SvcPntGUID {F243DB28-4D30-460A-946B-BE76EC9DA7D1} Customer Consumption 1755.0
SvcPntGUID {D91E5390-CFEB-431F-BCCF-983F4DB1B1E4} Customer Consumption 2603.0
SvcPntGUID {233736A5-FB49-4604-89A7-EFC08938B277} Customer Consumption 2361.0
SvcPntGUID {CAE20695-71DF-480E-877A-05EDD0606125} Customer Consumption 1726.0
SvcPntGUID {D041CCA2-44AF-460D-940C-E41F30E811C8} Customer Consumption 1971.0
SvcPntGUID {C6D6489D-77C8-40C1-8475-EB92A9CA2D05} Customer Consumption 1074.0
SvcPntGUID {F75B9CC2-BB87-4E17-B98C-DB0551B21A32} Customer Consumption 1584.0
SvcPntGUID {96F02D72-C97D-4434-8D3E-2BE6A64D2A5B} Customer Consu

SvcPntGUID {303FF1FD-8BD4-4AB7-9FF4-A2D25B104705} Customer Consumption 2146.0
SvcPntGUID {84D6A9B1-AB37-40C2-885F-106FA8964FBC} Customer Consumption 1642.0
SvcPntGUID {08232384-C935-4C92-8834-A3B69BADD608} Customer Consumption 2494.0
SvcPntGUID {BDE88AA3-38BD-4BC3-969C-DEEA4F8BD1FC} Customer Consumption 2066.0
SvcPntGUID {9991D739-C70E-4FBD-AD5B-383428E74730} Customer Consumption 87.0
SvcPntGUID {B5754158-80B8-4E1A-873A-2C2FC5AC2911} Customer Consumption 1411.0
SvcPntGUID {C45445FD-4E21-44BD-9AEA-A1EF8C6A71EE} Customer Consumption 1780.0
SvcPntGUID {FD43E7FF-6DB2-4B2B-A76D-0F07E3DAD42D} Customer Consumption 30.0
SvcPntGUID {0FA4D095-DB8D-4065-B2B5-6CC8BDC64966} Customer Consumption 2586.0
SvcPntGUID {7C737226-32FD-47AF-96FF-E31DDE5FCC16} Customer Consumption 1794.0
SvcPntGUID {2BDEE14B-28DE-4D7C-9AAC-8D73088EFAB7} Customer Consumption 543.0
SvcPntGUID {66A5FAA5-DAEC-4570-8CF3-E57A35A1FBCB} Customer Consumption 4324.0
SvcPntGUID {8F2A74B3-360D-481B-BED3-A454C8CD6FEF} Customer Consumptio

In [ ]:
edit.stopOperation()
edit.stopEditing(True)


# ### Use numpy and pandas to export to CSV
# 
# Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)
# See also [Working with numpy in ArcGIS](http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm)

# nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,skip_nulls=True)
# #nparr = arcpy.da.TableToNumPyArray(custAcctDest,custAcctOutFlds,null_value=-9999)
# pdarr = pd.DataFrame(nparr)
# pdarr.to_csv(custAcctFile,header=False, index=False)

# TODO - write file to \\gruadmin.gru.com\fs\Groups\OMS Replacement Project\Documents for OSII\Customer and Premise Files

# Ready new numpy array for consumption analysis

# nparr = arcpy.da.TableToNumPyArray(custAcctDest,["SERVICEPOINTOBJECTID","POINT_X","POINT_Y","AVGCONSUMPTION","MAXCONSUMPTION"],skip_nulls=True)

# df = pd.DataFrame(nparr)
# df.head()

# [```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

# from arcgis.features import SpatialDataFrame
# from arcgis.gis import GIS
# from getpass import getpass
# from IPython.display import display

# sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
# gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
# #gis = GIS()
# #portalDesc = arcpy.GetPortalDescription()
# # search and list all items owned by connected user
# #query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
# #itemType="Feature Layer"
# #sortField="title"
# #sortOrder="asc"
# # default max__items is 10
# #maxItems=100
# #m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)

# #consumptionLyr = gis.content.import_data(sdf)

# m = gis.map('Gainesville,FL')

# m.add_layer(sdf,options={"renderer":"ClassedSizeRenderer","field_name":"MAXCONSUMPTION"})

# #m.add_layer(consumptionLyr,options={"renderer":"ClassedSizeRenderer","field_name":"MAXCONSUMPTION"})

# m
